<a href="https://colab.research.google.com/github/rpatel71/Design-Optimization/blob/main/homework_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the answer of the homework-3 of Design Optimization.

Problem 1

In [2]:
# importing all the libraries required
import numpy as np
import torch as tor
from torch.autograd import Variable
import matplotlib.pyplot as plt

X1 = np.array([[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]])
X2 = np.flip(X1, axis = 1).copy()
a = np.array(([[8.0731, 1730.63, 233.426], [7.43155, 1554.679, 240.337]]))

# T = 20
p_w = 10 ** (a[0, 0] - a[0, 1] / (20 + a[0, 2]))
p_diox = 10 ** (a[1, 0] - a[1, 1] / (20 + a[1, 2]))
P = np.array([[28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5]])
P = tor.tensor(P, requires_grad=False, dtype=tor.float32)

A = Variable(tor.tensor([2.0, 2.0]), requires_grad=True)

x1 = tor.tensor(X1, requires_grad=False, dtype=tor.float32)
x2 = tor.tensor(X2, requires_grad=False, dtype=tor.float32)

a = 0.001

for i in range(100):
    P_cal = x1 * tor.exp(A[0] * (A[1] * x2 / (A[0] * x1 + A[1] * x2)) ** 2) * p_w + \
          x2 * tor.exp(A[1] * (A[0] * x1 / (A[0] * x1 + A[1] * x2)) ** 2) * p_diox

    loss = (P_cal - P) ** 2
    loss = loss.sum()

    loss.backward()

    with tor.no_grad():
        A -= a * A.grad
        A.grad.zero_()

print('estimation of A12 and A21 is: ', A)
print('final loss is: ', loss.data.numpy())

estimation of A12 and A21 is:  tensor([1.9531, 1.6834], requires_grad=True)
final loss is:  0.67093503


Problem 2

In [3]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=766b87ac023459851d5a23c24bb86e6077943151218d7b165715256627796a0f
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [7]:
from bayes_opt import BayesianOptimization
def eq(x1, x2):
  var = -((4 - (2.1*x1**2) + ((x1**4)/3))*x1**2 + (x1*x2) + (-4 + 4*(x2**2)) * x2**2)
  # we are taking the var as "negative" because this library only have maximize function.
  var_flip = -var
  return var
lim = {'x1': (-3, 3), 'x2': (-2, 2)}

opt = BayesianOptimization(f=eq, pbounds=lim, random_state=1)

# To minimize the function required, we need to use maximize function since we have already taken the function as negative.
opt.maximize(init_points = 10, n_iter = 101)
print(opt.max)

|   iter    |  target   |    x1     |    x2     |
-------------------------------------------------
|  1        |  0.265    | -0.4979   |  0.8813   |
|  2        | -110.1    | -2.999    | -0.7907   |
|  3        | -26.91    | -2.119    | -1.631    |
|  4        | -2.856    | -1.882    | -0.6178   |
|  5        | -1.054    | -0.6194   |  0.1553   |
|  6        |  0.5211   | -0.4848   |  0.7409   |
|  7        | -11.28    | -1.773    |  1.512    |
|  8        | -66.76    | -2.836    |  0.6819   |
|  9        | -0.5377   | -0.4962   |  0.2348   |
|  10       | -12.02    | -2.158    | -1.208    |
|  11       | -6.707    | -0.6913   | -1.287    |
|  12       | -4.507    |  1.223    |  1.081    |
|  13       | -0.5198   |  1.354    | -0.6292   |
|  14       | -65.32    |  2.812    |  0.1161   |
|  15       | -48.23    |  1.016    | -2.0      |
|  16       | -49.11    |  0.3388   |  2.0      |
|  17       | -1.532    |  0.7023   |  0.103    |
|  18       | -68.92    |  2.385    |  2.0      |


In [9]:
# Here we are multiplying the target value with (-1) since we found the value of negative of the objective function
target = -1*opt.max['target']
min = opt.max['params']
print(f'The function reaches value of {target} at the minimum point {min}')

The function reaches value of -1.0315328891000553 at the minimum point {'x1': 0.08506691846494913, 'x2': -0.7133078530616515}
